In [1]:
import pandas as pd, os 
from google.cloud import bigquery
from tqdm import tqdm

In [2]:
from dotenv import load_dotenv
load_dotenv("/Users/juanluis/Documents/credentials/env.txt")

True

### Loads newspaper articles to BQ

In [7]:
bq_client = bigquery.Client()

In [3]:
stocks = ["AAPL", 'KHC', 'PEP', 'GS', 'MSFT', 'MCD', 'OXY']
gcloud_project = "stock-sentiment-nlp"
news = {}

In [4]:
"""
## Loads data from csv to BQ
# pyarrow needs to be installed to load df to tables
news_df = {}
path = "./news_scraping/csv"
for stock in os.listdir(path):
    if os.path.isdir(f"{path}/{stock}"):
        months_df = []
        for file in os.listdir(f"{path}/{stock}"):
            month = pd.read_csv(f"{path}/{stock}/{file}", index_col = 0)
            month.date_google = pd.to_datetime(month.date_google, errors = "coerce")
            month.date_article = pd.to_datetime(month.date_article, errors = "coerce")
            months_df.append(month)
        news_df[stock] = pd.concat(months_df).drop_duplicates("title").sort_values("date_google").dropna()

for stock, data in news_df.items():
    data.insert(0,"id", list(map(str,data.index.values)))
    bq_client.load_table_from_dataframe(
        dataframe = data,
        destination = f"{gcloud_project}.news.{stock}_news"
    )
    print(f"Table for {stock} news created")
    
for stock, data in news_df.items():
    print(stock, data.shape)

"""

'\n## Loads data from csv to BQ\n# pyarrow needs to be installed to load df to tables\nnews_df = {}\npath = "./news_scraping/csv"\nfor stock in os.listdir(path):\n    if os.path.isdir(f"{path}/{stock}"):\n        months_df = []\n        for file in os.listdir(f"{path}/{stock}"):\n            month = pd.read_csv(f"{path}/{stock}/{file}", index_col = 0)\n            month.date_google = pd.to_datetime(month.date_google, errors = "coerce")\n            month.date_article = pd.to_datetime(month.date_article, errors = "coerce")\n            months_df.append(month)\n        news_df[stock] = pd.concat(months_df).drop_duplicates("title").sort_values("date_google").dropna()\n\nfor stock, data in news_df.items():\n    data.insert(0,"id", list(map(str,data.index.values)))\n    bq_client.load_table_from_dataframe(\n        dataframe = data,\n        destination = f"{gcloud_project}.news.{stock}_news"\n    )\n    print(f"Table for {stock} news created")\n    \n        \n\n'

In [8]:
#Downloads data from BQ to pandas
for stock in tqdm(stocks):
    df = bq_client.query(
        f"""SELECT date_google, date_article, title, description
            FROM `{gcloud_project}.news.{stock}_news`"""
            ).result().to_dataframe()
    news[stock] = df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:22<00:00,  3.26s/it]


## Extract sentiment from titles

In [ ]:
from transformers import pipeline, AutoTokenizer

In [ ]:
nlp = pipeline("sentiment-analysis", 
               model = "nlptown/bert-base-multilingual-uncased-sentiment", 
               return_all_scores = True,
              )

In [ ]:
nlp_binary = pipeline("sentiment-analysis", return_all_scores = True)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
for stock, data in news_df.items():
    dates = pd.DataFrame(index = pd.date_range(start = "2019-01-01", end = "2020-12-31"))
    group = data[["id","date_google"]].groupby("date_google").count()
    series = pd.merge(dates,group, right_index=True,left_index=True, how = "left")
    print(stock, series.isna().sum()["id"], series.isna().sum()["id"]/dates.shape[0])


In [ ]:
aapl.df.filter(regex= ".*star|date").groupby("date_google").mean()

In [ ]:
sentiment
- date
- avg sent pred 1,2,3,4,5
- close, open, etc

headlines
- date
- id
- classif (0-5)
- url
- title
- description


In [ ]:
predict(target)

In [ ]:
for stock in ["AAPL", 'KHC', 'PEP', 'GS', 'MSFT', 'MCD', 'OXY']:
    sp = SentimentPredictor(stock)
    sp.predict_title_and_description()
    sp.push_headlines_to_bq()
    sp.push_series_to_bq()
    print("STOCK COMPLETED", stock, datetime.now())
    sentiment_predictors[stock] = sp
    

In [ ]:
class SentimentPredictor:
    gcloud_project = "stock-sentiment-nlp"
    bq_client = get_bq_client(os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"), gcloud_project)
    nlp = pipeline("sentiment-analysis", 
               model = "nlptown/bert-base-multilingual-uncased-sentiment",
               return_all_scores = True)
    
    def __init__(self,stock):
        self.stock_name = stock
        self.get_df_from_bq()
    
    def get_df_from_bq(self):
        self.df = bq_client.query(
            f"""SELECT id, date_google, title, description, url 
                FROM `{gcloud_project}.news.{self.stock_name}_news`""").result().to_dataframe()
    
    def predict(self, text):
        while(len(text) > 0):
            try:
                return self.nlp(text)
            except RuntimeError:
                text = " ".join(text.split(" ")[:-100])
        
    def predict_and_separate_stars(self,x):
        out = self.predict(x)[0]
        return out[0]["score"], out[1]["score"], out[2]["score"], out[3]["score"], out[4]["score"]
    
    def predict_title_and_description(self):
        text = self.df.title.apply(str) #+ " " + self.df.description.apply(str)
        self.df['star1'], self.df['star2'], self.df['star3'], self.df['star4'], self.df['star5'] = zip(*text.map(self.predict_and_separate_stars))
        self.df["winner"] = self.df.filter(regex = ".*star").idxmax(axis=1)
        
    
    def push_headlines_to_bq(self):
        self.headlines = self.df[["date_google", "url", "title", "description", "winner"]]
        bq_client.load_table_from_dataframe(
            dataframe = self.headlines,
            destination = f"{gcloud_project}.headlines.{self.stock_name}_titles"
            )
    def push_series_to_bq(self):
        dates = pd.DataFrame(index = pd.date_range(start = "2019-01-01", end = "2020-12-31"))
        average = self.df.filter(regex= "star.*|date").groupby("date_google").median()
        average.index = average.index.tz_localize(None)
        self.series = pd.merge(dates,average, right_index=True,left_index=True, how = "left").reset_index()
        bq_client.load_table_from_dataframe(
            dataframe = self.series,
            destination = f"{gcloud_project}.series.{self.stock_name}_series_titles_median"
            )
        
        
        
        
        
        
        